# Estimate $p$ with 10k Measurements in 5 Seconds

Key features:
- Robustness:
    - Works even if numerical integration fails or the system is chaotic;
    - Handles measurements with unequal time steps;
- Effectiveness: 
    - Allow less shooting nodes to reduce the complexity (and it does work);
    - Parallel shooting;
    - Schur complement for sparse QP in Gauß-Newton;
    - `ipopt` with precomputed $H$ and `JIT`

## Plan vs Actual

**Plan**: One week per Milestone.

**Actual**:
- Week 1: 
    - Environment
    - Papers
    - Code framework
    - Start `_build_cnlls` and `_solve_ipopt`
- Week 2:
    - Start `interface.py`
    - Start `_solve_gn`
- Week 3: 
    - Gauss-Newton sometimes does not converge, add line search
    - Start `demo.ipynb`
- Week 4: Realized that shooting can be **parallel** and $H$ is **sparse**
    - Refactored `_build_cnlls` 
    - Add Schur complement in `_solve_gn`

## Some demo

In [ ]:
import numpy as np
import dms_estimator as dmse
from matplotlib import pyplot as plt

%matplotlib inline
plt.rc('figure', dpi=100)
np.set_printoptions(linewidth=100, suppress=True, precision=2, floatmode='fixed')

###  Effectiveness of Multiple Shooting

Notorious test in §6 of [[Bock07](https://onlinelibrary.wiley.com/doi/pdf/10.1002/gamm.200790024?casa_token=1vkAqyG6JuwAAAAA%3AGwUN3aX6DOsp_FOatMYMQSZCDm27CWARJg90nhxYuZE_UCYlOsUxKim3UcWdrZjMtffexjzbiOGRJA)]

ODE:
$$
\dot{x}_1=x_2 \quad \dot{x}_2=\mu^2 x_1-\left(\mu^2+p^2\right) \sin p t \quad t \in[0,1]
$$

Initial values:
$$
x_1(0)=0 \quad x_2(0)=\pi
$$

The solution for the true parameter value $p=\pi$

$$
x_1(t)=\sin \pi t, \quad x_2(t)=\pi \cos \pi t
$$

In [ ]:
cfg = dmse.ModelConfig(
        name="Notorious",
        build_integrator=dmse.notorious_problem,
        true_p=[np.pi],
        x0=np.array([0.0, np.pi, 0.0]),
        p_init=[1.0],
        state_labels=["x1", "x2", "t"],
)

Let's see what happens when we do numerical integration from $0$

In [ ]:
dt = 0.01
t_grid = np.arange(0.0, 1.0, dt)

integrator, ode, states, params = cfg.build_integrator(dt)
X_true, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=0.0)
plt.plot(t_grid, X_true[:, 0], label='cvodes solution')

dt = 0.01
t_grid = np.arange(0.0, 1.0, dt)

X_true = np.array([
    np.sin(np.pi * t_grid), 
    np.pi * np.cos(np.pi * t_grid), 
    t_grid]
    ).T
rng = np.random.default_rng(42)
plt.plot(t_grid, X_true[:, 0], label='analytic solution')

plt.ylabel("$x_1$")
plt.ylim(0, 2)
plt.legend()
plt.show()

In [ ]:
dt = 0.1
t_grid = np.arange(0.0, 1.0, dt)

X_true = np.array([
    np.sin(np.pi * t_grid), 
    np.pi * np.cos(np.pi * t_grid), 
    t_grid]
    ).T
rng = np.random.default_rng(42)
X_meas = X_true + 0.01 * rng.standard_normal(X_true.shape)

In [ ]:
print("[Single Shooting]")
p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_true,
    cfg.p_init,
    num_shooting=1,
    strategy="gn_fast",
)
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

Single shooting failed, try multiple shooting

In [ ]:
p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_meas,
    cfg.p_init,
    num_shooting=len(t_grid),
    strategy="gn",
)
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

Even for **chaotic** dynamic system with **huge** noise!

In [ ]:
cfg = dmse.ModelConfig(
        name="Lorenz",
        build_integrator=dmse.lorenz_problem,
        true_p=[10.0, 28.0, 8.0/3.0],
        x0=np.array([3.0, 1.5, 15.0]),
        p_init=[10.0] * 3,
        state_labels=["x", "y", "z"],
)

dt = 0.02
t_grid = np.arange(0.0, 10.0, dt)

integrator, ode, states, params = cfg.build_integrator(dt)
X_true, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=3.0)

p_hat, s_hat = dmse.estimate(
    ode,
    states,
    params,
    t_grid,
    X_meas,
    cfg.p_init,
    num_shooting=200,
    strategy="gn_fast",
)
dmse.plot(t_grid, X_meas[:, :2], s_hat[:, :2], cfg.state_labels[:2], true=X_true[:, :2])
plt.show()
print(f"Estimated:\n{p_hat}")
print(f"True:\n{np.array(cfg.true_p)}")

### Solve the pyridine problem

In [ ]:
from ipywidgets import interact, IntSlider, Dropdown

cfg = dmse.ModelConfig(
        name="Pyridine",
        build_integrator=dmse.pyridine_problem,
        true_p=[1.81, 0.894, 29.4, 9.21, 0.058, 2.43, 0.0644, 5.55, 0.0201, 0.577, 2.15],
        x0=np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        p_init=[1.0] * 11,
        state_labels=list("ABCDEFG"),
)

def run_estimation(num_meas, num_shooting, strategy):
    print(f"Start with (num_meas={num_meas}, num_shooting={num_shooting}, strategy='{strategy}'):")
    T = 5.0
    dt = T / num_meas
    t_grid = np.arange(0.0, T, dt)

    integrator, ode, states, params = cfg.build_integrator(dt)
    _, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p)
    # Perform estimation
    p_hat, s_hat = dmse.estimate(
        ode,
        states,
        params,
        t_grid,
        X_meas,
        cfg.p_init,
        num_shooting=num_shooting,
        strategy=strategy,
    )
    # Plot results
    dmse.plot(t_grid, X_meas, s_hat, cfg.state_labels, show_every=5)
    plt.show()
    print(p_hat)
    print("True parameters:")
    print(np.array(cfg.true_p))

# Create interactive sliders and dropdown
_ = interact(
    run_estimation,
    num_meas=IntSlider(
        value=500,
        min=1,
        max=500,
        step=1,
        description='#Meas',
        continuous_update=False
    ),
    num_shooting=IntSlider(
        value=50,
        min=1,
        max=500,
        step=1,
        description='#Node',
        continuous_update=False
    ),
    strategy=Dropdown(
        options=['ipopt', 'gn', 'gn_fast'],
        value='gn',
        description='Strategy'
    )
)

$p_4$ is the coefficient of $C^2$ and $p_8, p_{11}$ if the coefficient of E

$C$ and $E$ are almost $0$, so larger error for these three.

## Method

TODO:
- shooting(parallel)
- GN
- Schur
- IG

## Benchmark Test

Time(s) consumed for different \#measurements, \#nodes and strategies(from `benchmark.py`)

![talbe](./results/table.svg)
\*all converge to acceptable result

where
- **IP** is `ipopt`
- **IG** is `ipopt` with the given $H = J^\top J$ and `JIT`
- **GN** is the Gauß-Newton we implement with Schur complement.

**Findings**:
- **GN**: Gauß-Newton is optimal when the `#node` is small;
- **IPs**: The complexity of the `ipopt` does not increase significantly with the number of decision variables;
- **IG**: Outstanding in large-scale. Possible reasons:
    1. Using $J^\top J$ saves time in computing the Hessian and is good enough in least squares problems;
    2. `JIT` compiles the solver firstly thus saving time for each iteration.

**Problems**:
- I don't think the complexity of `qpoases` here should be $\mathcal{O}(n^2)$, but time-wise it's close to this.